## AGN in Extended Chandra Deep Field South (ECDFS)

- author : Sylvie Dagoret-Campagne with the help of ChatGPT 4
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2025-04-20
- last update : 2025-04-21

In [ ]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np

## Selection of a catalog in ECDFS
- https://heasarc.gsfc.nasa.gov/docs/heasarc/biblio/pubs/chandra_ycat.html

In [ ]:
ECDFS_RA = 53.1
ECDFS_DEC = -27.8

In [ ]:
# Coordonnées approximatives du centre du E-CDFS
coord = SkyCoord(ra=ECDFS_RA, dec=ECDFS_DEC, unit=(u.deg, u.deg), frame='icrs')
radius = 15 * u.arcmin

# Exemple avec le catalogue Luo+2017 (7Ms CDF-S)
catalog_id = "J/ApJS/228/2"

result = Vizier(columns=["*"]).query_region(coord, radius=radius, catalog=catalog_id)

- **sources** : c’est la table principale, avec les propriétés astrophysiques des sources (coordonnées, flux X, luminosité, classification AGN potentielle, etc.)
- **table1** : généralement des infos complémentaires ou des index d’identifiants croisés (par ex. index interne, flags de détection, etc.)

In [ ]:
result

In [ ]:
# Voir les résultats
result[0]

In [ ]:
result[1]

## Selection of AGN

In [ ]:
sources_table = result['J/ApJS/228/2/sources']
print(sources_table.colnames)

In [ ]:
# On calcule le log10 de LX
logLX = np.log10(sources_table['LX'])

# Et on l'ajoute comme nouvelle colonne à la table
sources_table['logLX'] = logLX

In [ ]:
# Exemple : filtrer les sources avec LX > 10^42 erg/s
import numpy as np

# Attention au nom exact de la colonne
logLX = sources_table['logLX']
is_agn = logLX > 42

In [ ]:
is_agn = sources_table['logLX'] > 42  # AGN si LX > 10^42 erg/s
agn_table = sources_table[is_agn]

In [ ]:
agn_table[['RAJ2000', 'DEJ2000', 'logLX', 'LX', 'zspec']].pprint(max_lines=10)

### Save agn table

In [ ]:
df_agn = agn_table.to_pandas()

In [ ]:
df_agn.to_csv("agn_ecdfs.csv") 

## Plot on AGN

## 🌍 1. Tracer logLX vs redshift (zspec)
Pour explorer la distribution des AGN selon leur luminosité X et leur redshift :

In [ ]:
import matplotlib.pyplot as plt

# Ne garder que les objets avec zspec connu
has_zspec = ~sources_table['zspec'].mask  # ou sources_table['zspec'].filled(-1) > 0

plt.figure(figsize=(8,6))
plt.scatter(sources_table['zspec'][has_zspec], sources_table['logLX'][has_zspec], 
            c='blue', alpha=0.7, edgecolors='k')
plt.xlabel("Redshift spectroscopique (zspec)")
plt.ylabel("log(Lx) [erg/s]")
plt.title("Distribution des AGN dans l'E-CDFS")
plt.grid(True)
plt.show()

## 📊 2. Histogrammes pour explorer la population
Distribution en redshift

Distribution en luminosité

In [ ]:
plt.figure(figsize=(14,5))

plt.subplot(1, 2, 1)
plt.hist(sources_table['zspec'][has_zspec], bins=20, color='green', alpha=0.7)
plt.xlabel("Redshift zspec")
plt.ylabel("Nombre de sources")
plt.title("Distribution en redshift")

plt.subplot(1, 2, 2)
plt.hist(sources_table['logLX'], bins=20, color='orange', alpha=0.7)
plt.xlabel("log(Lx) [erg/s]")
plt.ylabel("Nombre de sources")
plt.title("Distribution de la luminosité X")

plt.tight_layout()
plt.show()


## 🌌 3. Diagramme couleur (ex: BR vs logLX)
Si BR est une couleur optique (par exemple B-R), tu peux chercher des relations avec l’activité nucléaire :

In [ ]:
plt.scatter(sources_table['BR'], sources_table['logLX'], c='purple', alpha=0.7)
plt.xlabel("B-R (mag)")
plt.ylabel("log(Lx) [erg/s]")
plt.title("Couleur optique vs Luminosité X")
plt.grid(True)
plt.show()

## 🧭 4. Carte du ciel des AGN

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(sources_table['RAJ2000'], sources_table['DEJ2000'], 
            s=20, c='gray', label="Toutes les sources")
plt.scatter(agn_table['RAJ2000'], agn_table['DEJ2000'], 
            s=40, c='red', label="AGN (logLX > 42)")
plt.xlabel("RA (deg)")
plt.ylabel("DEC (deg)")
plt.title("Carte du ciel des sources E-CDFS")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
assert False

## 🔗 5. Croisement avec un autre catalogue (optique, IR, radio...)
Si tu as un autre catalogue (par exemple CANDELS, GOODS-S, VLA...), on peut faire un cross-match spatial basé sur les coordonnées RA/DEC (avec astropy.coordinates.SkyCoord).

Exemple possible :

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u

cat1 = SkyCoord(ra=sources_table['RAJ2000'], dec=sources_table['DEJ2000'], unit='deg')
cat2 = SkyCoord(ra=autre_table['RA'], dec=autre_table['DEC'], unit='deg')

idx, d2d, _ = cat1.match_to_catalog_sky(cat2)
matched = autre_table[idx[d2d < 1.0*u.arcsec]]




    from lsst.daf.butler import Butler
    from astropy.coordinates import SkyCoord
    import astropy.units as u

    # Initialisation du dépôt LSST
    butler = Butler("/repo/main")  # à adapter à ton dépôt local, ou celui de DP0

    # Tu peux aussi voir quelles collections sont disponibles :
    print(butler.registry.queryCollections())

    # Ex: sélectionner un sky region (coordonnées du champ E-CDFS)
    center = SkyCoord(ra=53.1*u.deg, dec=-27.8*u.deg)
    radius = 5.0 * u.arcmin

    # Trouver quels tracts couvrent cette région
    tracts = butler.registry.queryRegion("deepCoadd", region=center, radius=radius)

    for tract in tracts:
        # Accès aux catalogues d’objets détectés dans ce tract
        data_refs = butler.registry.queryDatasets("deepCoadd_forced_src", tract=tract.tract, band='i')

        for ref in data_refs:
            cat = butler.get(ref)
            # Croisement spatial ici avec ton catalogue Chandra
    🚀 Tu travailles :
    avec un repo local ComCam, sur une instance personnelle ?

    ou bien tu accèdes aux données DP0.x via la Rubin Science Platform (RSP) en ligne ?

    ou tu installes LSST pipelines en local pour t’entraîner ?

    Je peux t’aider à :

    initialiser un dépôt de test ComCam ou DP0

    écrire une fonction de croisement Chandra ↔ LSST objets détectés

    extraire automatiquement les flux LSST associés à tes AGN

    Tu veux faire ça depuis un notebook local ? ou via le portail Rubin Science Platform (JupyterHub sur https://data.lsst.cloud) ?









